## Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import pandas as pd
import pickle as pkl
from datetime import datetime
from dateutil import parser
from datetime import datetime
from sys import getsizeof
import re

import operator
from scipy import stats
import scipy.optimize as optimization

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
plt.rcParams['figure.figsize'] = (12.0, 8.0)
plt.rcParams['axes.axisbelow'] = True

from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.express as px
init_notebook_mode(True)

import seaborn as sns

from wand.image import Image as WImage

from hallprobecalib import hpc_ext_path
from hallprobecalib.hpcplots import scatter2d,scatter3d,histo,py_profile

In [4]:
plotdir = hpc_ext_path+'plots/hall_probe_stability/2019-12-04/'

In [5]:
pkl_dir = hpc_ext_path+"datafiles/hall_probe_stability/"
pkl_file_slow = "hall_stability.run06.raw.slow.pkl"
#pkl_file_fast = "hall_stability.run06.raw.fast.pkl"
#pkl_file_fast_sparse = "hall_stability.run06.sparse.fast.pkl"

In [6]:
df = pd.read_pickle(pkl_dir+pkl_file_slow)

In [7]:
#df_fast = pd.read_pickle(pkl_dir+pkl_file_fast)
#df_fast_sparse = pd.read_pickle(pkl_dir+pkl_file_fast_sparse)

In [8]:
#df_fast['seconds_delta'].iloc[60:75]

In [9]:
df['seconds_delta'].head(10)

Datetime
2019-12-04 14:38:18      0.0
2019-12-04 14:39:08     50.0
2019-12-04 14:39:58    100.0
2019-12-04 14:40:48    150.0
2019-12-04 14:41:38    200.0
2019-12-04 14:42:28    250.0
2019-12-04 14:43:18    300.0
2019-12-04 14:44:09    351.0
2019-12-04 14:44:58    400.0
2019-12-04 14:45:48    450.0
Name: seconds_delta, dtype: float64

In [10]:
def pplot(fig):
    pio.write_image(fig, 'temp.pdf')
    img = WImage(filename='temp.pdf')
    return img

## Cleaning

In [11]:
directory = "/home/ckampa/Dropbox/LogFiles/"#hpc_ext_path+'datafiles/magnet_stability/nmr/'
filename_slow = '2019-12-04 143741slow.txt'#'2019-12-03 162209slow.txt' ## NO NMR IN FIRST FILE

In [12]:
df = pd.read_csv(directory+filename_slow)

In [ ]:
df

In [13]:
df.columns = df.columns.str.lstrip(" ")

In [14]:
df.drop("", axis=1, inplace=True)

In [ ]:
#df["NMR [T]"]

In [15]:
dates = [parser.parse(row.Time) for row in df.itertuples()]# if isdatetime(row.Time.values[0])]

In [16]:
df["Datetime"] = pd.to_datetime(dates)#.astype("str")
df = df.set_index("Datetime")

In [ ]:
## PLOT ALL
#for col in df.columns:
#    fig = scatter2d(df.index, df[col], show_plot=False)
#    fname = f"{plotdir}time/all/{col}_vs_time"
#    pio.write_image(fig, fname+".pdf")
#    pio.write_image(fig, fname+".png")

In [17]:
probes = np.array([])
for col in df.columns:
    #print(df_fast[col].dtype)
    if ("Raw" in col) or ("Cal" in col):#type(df[col].iloc[0]) is np.float64:
        #print(f"{col}")#": mean: {df[col].mean()}")
        probes = np.append(probes, col[:16])
probes = list(np.unique(probes))
print(probes)

['6A0000000D61333A', 'C50000000D5E473A', 'DF0000000D5E803A']


In [18]:
probe = probes[2]

In [ ]:
probe

In [ ]:
for col in df.columns:
    #print(df_fast[col].dtype)
    if type(df[col].iloc[0]) is np.float64:
        print(f"{col}: mean: {df[col].mean()}")

- Looks like maybe probe "DF0000000D5E803A" is in the field
- NMR [T] = 0?

### Slow Control Data Cleaning

In [19]:
for probe in probes:
    for t in ["Cal", "Raw"]:
        df[f"{probe}_{t}_Bmag"] = (df[f"{probe}_{t}_X"]**2+df[f"{probe}_{t}_Y"]**2+df[f"{probe}_{t}_Z"]**2)**(1/2)
        df[f"{probe}_{t}_Res"] = df[f"{probe}_{t}_Bmag"] - df['NMR [T]']
        df[f"{probe}_{t}_Res_Rel"] = (df[f"{probe}_{t}_Bmag"] - df['NMR [T]']) / df['NMR [T]']

In [20]:
for B in ["X", "Y", "Z"]:
    for probe in probes:
        for t in ["Cal", "Raw"]:
            #df[f"{probe}_{t}_Bmag"] = (df[f"{probe}_{t}_X"]**2+df[f"{probe}_{t}_Y"]**2+df[f"{probe}_{t}_Z"]**2)**(1/2)
            df[f"{probe}_{t}_{B}_Res"] = df[f"{probe}_{t}_{B}"] - df['NMR [T]']
            df[f"{probe}_{t}_{B}_Res_Rel"] = (df[f"{probe}_{t}_{B}"] - df['NMR [T]']) / df['NMR [T]']

In [ ]:
## PLOT HISTOS
for probe in probes:
    for t in ["Cal", "Raw"]:
        fig = histo(df[f"{probe}_{t}_Bmag"], bins=50, show_plot=False)
        pio.write_image(fig, plotdir+f"histo/{probe}_{t}_Bmag.pdf")
        pio.write_image(fig, plotdir+f"histo/{probe}_{t}_Bmag.png")

In [ ]:
## RES HISTOS
for t in ["Cal", "Raw"]:
    for probe in probes:
        for B in ["X","Y","Z"]:
            fig = histo(df[f"{probe}_{t}_{B}_Res"], bins=50, show_plot=False)
            pio.write_image(fig, plotdir+f"histo/residuals/component/{probe}_{t}_{B}_Res.pdf")
            pio.write_image(fig, plotdir+f"histo/residuals/component/{probe}_{t}_{B}_Res.png")

In [ ]:
## REL_RES HISTOS
for t in ["Cal", "Raw"]:
    for probe in probes:
        for B in ["X","Y","Z"]:
            fig = histo(df[f"{probe}_{t}_{B}_Res_Rel"], bins=50, show_plot=False)
            pio.write_image(fig, plotdir+f"histo/relative_residuals/component/{probe}_{t}_{B}_Res_Rel.pdf")
            pio.write_image(fig, plotdir+f"histo/relative_residuals/component/{probe}_{t}_{B}_Res_Rel.png")

In [ ]:
## BMAG RES HISTOS
for res in ["Res", "Res_Rel"]:
    if res == "Res":
        p = "residuals"
    else:
        p = "relative_residuals"
    for t in ["Cal", "Raw"]:
        for probe in probes:
            #for B in ["X","Y","Z"]:
            fig = histo(df[f"{probe}_{t}_{res}"], bins=50, show_plot=False)
            pio.write_image(fig, plotdir+f"histo/{p}/{probe}_{t}_Bmag_{res}.pdf")
            pio.write_image(fig, plotdir+f"histo/{p}/{probe}_{t}_Bmag_{res}.png")

In [21]:
df["seconds_delta"] = (df.index - df.index[0]).total_seconds()
df["hours_delta"] = (df.index - df.index[0]).total_seconds()/60**2
df["days_delta"] = (df.index - df.index[0]).total_seconds()/(24*60**2)

In [22]:
type(df.days_delta) is not list

True

In [ ]:
# test new colors_list arg in scatter2d
fig = scatter2d(df.index, df["NMR [T]"], colors_list=df.days_delta, markers=True)

In [ ]:
pplot(fig)

In [ ]:
# cal

In [23]:
for t in ["Cal", "Raw"]:
    for probe in probes:
        df[f"{probe}_{t}_Transverse"] = (df[f"{probe}_Cal_X"]**2+df[f"{probe}_Cal_Y"]**2)**(1/2)
        df[f"{probe}_{t}_Theta"] = np.arccos(df[f"{probe}_Cal_Z"]/df[f"{probe}_Cal_Bmag"])
        df[f"{probe}_{t}_Phi"] = np.arccos(df[f"{probe}_Cal_X"]/df[f"{probe}_Cal_Transverse"])

In [ ]:
## PLOT HISTO ANGLE
for probe in probes:
    for angle in ["Transverse", "Theta", "Phi"]:
        fig = histo(df[f"{probe}_Cal_{angle}"], bins=30, show_plot=False)
        pio.write_image(fig, plotdir+f"histo/angles/{probe}_Cal_{angle}.pdf")
        pio.write_image(fig, plotdir+f"histo/angles/{probe}_Cal_{angle}.png")

In [ ]:
## PLOT ANGLE
for probe in probes:
    for angle in ["Transverse", "Theta", "Phi"]:
        fig = scatter2d(df.index, df[f"{probe}_Cal_{angle}"], show_plot=False)
        pio.write_image(fig, plotdir+f"time/angles/{probe}_Cal_{angle}.pdf")
        pio.write_image(fig, plotdir+f"time/angles/{probe}_Cal_{angle}.png")

In [24]:
# raw
for probe in probes:
    df[f"{probe}_Raw_Bmag"] = (df[f"{probe}_Raw_X"]**2+df[f"{probe}_Raw_Y"]**2+df[f"{probe}_Raw_Z"]**2)**(1/2)

for probe in probes:
    df[f"{probe}_Raw_Transverse"] = (df[f"{probe}_Raw_X"]**2+df[f"{probe}_Raw_Y"]**2)**(1/2)
    df[f"{probe}_Raw_Theta"] = np.arccos(df[f"{probe}_Raw_Z"]/df[f"{probe}_Raw_Bmag"])
    df[f"{probe}_Raw_Phi"] = np.arccos(df[f"{probe}_Raw_X"]/df[f"{probe}_Raw_Transverse"])

In [ ]:
## PLOT ANGLES RAW
for probe in probes:
    for angle in ["Transverse", "Theta", "Phi"]:
        fig = histo(df[f"{probe}_Raw_{angle}"], bins=30, show_plot=False)
        pio.write_image(fig, plotdir+f"histo/angles/{probe}_Raw_{angle}.pdf")
        pio.write_image(fig, plotdir+f"histo/angles/{probe}_Raw_{angle}.png")

for probe in probes:
    for angle in ["Transverse", "Theta", "Phi"]:
        fig = scatter2d(df.index, df[f"{probe}_Raw_{angle}"], show_plot=False)
        pio.write_image(fig, plotdir+f"time/angles/{probe}_Raw_{angle}.pdf")
        pio.write_image(fig, plotdir+f"time/angles/{probe}_Raw_{angle}.png")

In [25]:
df.to_pickle(pkl_dir+pkl_file_slow)

In [26]:
getsizeof(df) / 1000/1000

4.084602

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3198 entries, 2019-12-04 14:38:18 to 2019-12-06 11:02:32
Columns: 150 entries, Time to DF0000000D5E803A_Raw_Phi
dtypes: float64(149), object(1)
memory usage: 3.7+ MB


## Plotting

### First Attempt

In [28]:
len(df)

3198

In [29]:
len(df) * 50 / 60 / 60 / 24

1.8506944444444444

In [ ]:
#fig = scatter2d(df.index, df["NMR [T]"], show_plot=False);

In [ ]:
#pplot(fig)

In [ ]:
#fig = scatter2d(df.index, df["NMR [T]"])

In [ ]:
#plot(fig)

In [ ]:
fig = scatter2d(2*[df.index], [df["NMR [T]"], df[f"{probe}_Cal_Bmag"]], show_plot=False);

In [ ]:
plot(fig)

In [ ]:
#fig = scatter2d(df.index, df["Chamber Wall CH13"])

In [ ]:
fig = scatter2d(df.index, df[f"{probe}_Cal_Res_Rel"], show_plot=False);

In [ ]:
plot(fig)

~ -5e-5 change in residual after changing temperature (+4 deg C).

In [ ]:
fig = histo(df[f"{probe}_Cal_Res_Rel"], bins=75, show_plot=False);
plot(fig)
#fig = histo(df[f"{probe}_Cal_Res_Rel"], bins=50, yscale='log')

In [ ]:
df["day"] = df.index.day

In [ ]:
fig = px.histogram(df, x=f"{probe}_Cal_Res_Rel", color="day")

In [ ]:
fig.update_layout(
    autosize=False,
    width=1800,
    height=800
    #margin=dict(l=20, r=20, t=20, b=20),
    #padding=dict(l=20, r=20, t=20, b=20),
    #paper_bgcolor="LightSteelBlue",
);

In [ ]:
#fig.show()

In [ ]:
plot(fig)

In [ ]:
# temps

In [ ]:
temps_ch = [a for a in df.columns if ("CH" in a) and int(a[-2:])<21]# or "HVAC" in a]
temps_ch.append("Hall Element")
temps_ch.append("Floor")
temps_ch.append("Roof")
temps_ch.append("Yoke")
temps_ch.append("Yoke (center magnet)")

In [ ]:
#temps_ch

In [ ]:
#df[temps_ch].describe()

In [ ]:
#df[temps_ch].plot(kind="box")

In [ ]:
#fig = go.Figure()
#[fig.add_trace(go.Box(y=df[col], name=col)) for col in temps_ch]

#fig.show()

In [ ]:
fig = scatter2d([df.index for c in temps_ch],[df[c] for c in temps_ch],inline=True, show_plot=False);
fig.layout.title = "Temp vs. Datetime"
fig.layout.yaxis.title = "Temp [deg. C]";

In [ ]:
#fig.show()
#plot(fig)

In [ ]:
#fname = 'temps_scatter'
#fname = 'time/temps/temps_scatter_full'
#pio.write_image(fig, plotdir+fname+'.pdf')
#pio.write_image(fig, plotdir+fname+'.png')
#plot(fig, filename=plotdir+fname+'.html', auto_open=False)

In [ ]:
probe = probes[2]

In [ ]:
fig.add_trace(go.Scatter(x=df.index, y=df[f"{probe}_Cal_T"], mode="lines", name=f"{probe}_Cal_T"));

In [ ]:
#fname = 'temps_scatter'
fname = 'time/temps/temps_scatter_full'
pio.write_image(fig, plotdir+fname+'.pdf')
pio.write_image(fig, plotdir+fname+'.png')
plot(fig, filename=plotdir+fname+'.html', auto_open=False)

In [ ]:
plot(fig)

In [ ]:
temps_water = [col for col in df.columns if ("ICW" in col) or ("LCW" in col)]

In [ ]:
#temps_water

In [ ]:
fig = scatter2d([df.index for c in temps_water],[df[c] for c in temps_water],inline=True, show_plot=False);
fig.layout.title = "Water Temp vs. Datetime"
fig.layout.yaxis.title = "Temp [deg. C]"

In [ ]:
#fig.show()
plot(fig)

In [ ]:
#fname = 'temps_scatter'
fname = 'time/temps/temps_scatter_water'
pio.write_image(fig, plotdir+fname+'.pdf')
pio.write_image(fig, plotdir+fname+'.png')
plot(fig, filename=plotdir+fname+'.html', auto_open=False)

In [ ]:
temps_ambient = [col for col in df.columns if ("Ambient" in col)]
#temps_ambient

In [ ]:
fig = scatter2d([df.index for c in temps_ambient],[df[c] for c in temps_ambient],inline=True, show_plot=False);
fig.layout.title = "Ambient Temp vs. Datetime"
fig.layout.yaxis.title = "Temp [deg. C]"
#fig.show()
plot(fig)

In [ ]:
#fname = 'temps_scatter'
fname = 'time/temps/temps_scatter_ambient'
pio.write_image(fig, plotdir+fname+'.pdf')
pio.write_image(fig, plotdir+fname+'.png')
plot(fig, filename=plotdir+fname+'.html', auto_open=False)

In [ ]:
temps_magnet = [col for col in df.columns if (("PS" in col) or ("Coil" in col) or ("Yoke" in col)) and ("Orange" not in col)]
#temps_magnet

In [ ]:
fig = scatter2d([df.index for c in temps_magnet],[df[c] for c in temps_magnet],inline=True, show_plot=False);
fig.layout.title = "Magnet Temp vs. Datetime"
fig.layout.yaxis.title = "Temp [deg. C]"
#fig.show()
plot(fig)

In [ ]:
#fname = 'temps_scatter'
fname = 'time/temps/temps_scatter_magnet'
pio.write_image(fig, plotdir+fname+'.pdf')
pio.write_image(fig, plotdir+fname+'.png')
plot(fig, filename=plotdir+fname+'.html', auto_open=False)

## Seeing Temperature Corrections

In [ ]:
fig = scatter2d(df[f"{probe}_Cal_T"], df["NMR [T]"], df.days_delta, show_plot=False)

In [ ]:
plot(fig)

In [ ]:
fig = scatter2d(df[f"{probe}_Cal_T"], df[f"{probe}_Cal_Bmag"] / df["NMR [T]"], df.days_delta, inline=False)

In [ ]:
fig = scatter2d(df[f"{probe}_Cal_T"], df[f"{probe}_Raw_Bmag"] / df["NMR [T]"], df.days_delta, inline=False)

In [ ]:
fig = scatter2d(df[f"{probe}_Cal_T"], df[f"{probe}_Raw_Bmag"], df.days_delta, inline=False)

In [ ]:
fig = scatter2d(3*[df[f"{probe}_Cal_T"]], [df[f"{probe}_Raw_{comp}"] / df["NMR [T]"] for comp in ["X","Y","Z"]],
                3*[df.days_delta], inline=False)

In [ ]:
fig = scatter2d(df.index, df[f"{probe}_Raw_Theta"], df.days_delta, inline=False)

In [ ]:
fig = scatter2d(df.index, df[f"{probe}_Raw_Phi"], df.days_delta, inline=False)

In [30]:
def volt(temp, A, B):
    return A * temp + B

In [33]:
guess = np.array([-1300 / 3.5, 2e6]) # A, B

In [34]:
x_dat = df[f"{probe}_Cal_T"]
#x_dat = df[stable_temp][f"{probe}_Cal_T"]

In [35]:
params, cov = optimization.curve_fit(volt, x_dat, df[f"{probe}_Raw_Bmag"] / df["NMR [T]"], guess)
#params, cov = optimization.curve_fit(volt, x_dat, 
#                                     df[stable_temp][f"{probe}_Raw_Bmag"] / df[stable_temp]["NMR [T]"], guess)

In [ ]:
# Bmag ### STABLE TEMP
params, cov

In [36]:
# Bmag
params, cov

(array([-1.22617251e+03,  2.83351965e+06]), array([[ 0.24127382, -4.53719568],
        [-4.53719568, 85.99842428]]))

In [ ]:
# Bx
params, cov

In [ ]:
# By
params, cov

In [ ]:
# Bz
params, cov

In [37]:
np.sqrt(cov[0,0])

0.4911963170619433

In [38]:
cov[0,1] / (np.sqrt(cov[0,0])*np.sqrt(cov[1,1]))

-0.9960638093495933

Parameters have strong correlation

In [39]:
A_fit, B_fit = params

In [40]:
df["Bmag_lsq"] = volt(x_dat, A_fit, B_fit)

In [ ]:
fig = scatter2d(df[f"{probe}_Cal_T"], df[f"{probe}_Raw_Bmag"] / df["NMR [T]"], df.days_delta, show_plot=False)
#d = [df[f"{probe}_Raw_X"].min(), df[f"{probe}_Raw_X"].max()]
fig.add_trace(go.Scatter(
                    x=df[f"{probe}_Cal_T"], y=df["Bmag_lsq"],
                    mode='lines', name="unity",
                    line=dict(color='deeppink', width=2,
                              dash='dash')));
plot(fig)

In [ ]:
fig.update_traces(marker=dict(size=5));
plot(fig);

Bmag fit looks pretty good so far (only two data points...).

## Aggregate data

In [44]:
df["Floor"].diff(5).rolling(5).mean().head()

Datetime
2019-12-04 14:38:18   NaN
2019-12-04 14:39:08   NaN
2019-12-04 14:39:58   NaN
2019-12-04 14:40:48   NaN
2019-12-04 14:41:38   NaN
Name: Floor, dtype: float64

In [41]:
stable_temp = (abs(df["Floor"].diff(5).rolling(5).mean()) < 0.008) #| (np.isnan(df["Floor"].diff(5)))

In [42]:
fig = scatter2d(df[stable_temp][f"{probe}_Cal_T"],
                df[stable_temp][f"{probe}_Raw_Bmag"] / df[stable_temp]["NMR [T]"],
                df[stable_temp].days_delta, show_plot=False)
#d = [df[f"{probe}_Raw_X"].min(), df[f"{probe}_Raw_X"].max()]
fig.add_trace(go.Scatter(
                    x=df[stable_temp][f"{probe}_Cal_T"], y=df[stable_temp]["Bmag_lsq"],
                    mode='lines', name="unity",
                    line=dict(color='deeppink', width=2,
                              dash='dash')));
#plot(fig)

fig.update_traces(marker=dict(size=5));
plot(fig);

In [46]:
high_point = (df[f"{probe}_Cal_T"] > 20) & (df[f"{probe}_Cal_T"] < 21)
low_point = (df[f"{probe}_Cal_T"] > 16) & (df[f"{probe}_Cal_T"] < 17)
mid_point = (df[f"{probe}_Cal_T"] > 18) & (df[f"{probe}_Cal_T"] < 19)
h = high_point & stable_temp
l = low_point & stable_temp
m = mid_point & stable_temp

In [ ]:
plt.hist(df[h][f"{probe}_Raw_Bmag"]/df[h]["NMR [T]"], bins=50);

In [ ]:
fig = scatter2d(df[h][f"{probe}_Cal_T"], df[h][f"{probe}_Raw_Bmag"]/df[h]["NMR [T]"],
                df[h].days_delta, inline=False)

In [ ]:
hist2 = plt.hist2d(df[h][f"{probe}_Cal_T"], df[h][f"{probe}_Raw_Bmag"]/df[h]["NMR [T]"], bins=35);

In [ ]:
hist2[1]

In [ ]:
36*36

In [ ]:
len(hist2[1])

In [ ]:
len(hist2[2])

In [ ]:
int(np.floor(hist2[0].argmax() / 36))

In [ ]:
int(round(((hist2[0].argmax() / 35) - int(np.floor(hist2[0].argmax() / 35))) * 35))

In [ ]:
1296-int(np.floor(hist2[0].argmax() / 36))*36

In [ ]:
hist2[0][int(np.floor(hist2[0].argmax() / 35)), int(round(((hist2[0].argmax() / 35) - int(np.floor(hist2[0].argmax() / 35))) * 35))]

In [ ]:
for x, y in np.argwhere(hist2[0] == hist2[0].max()):
    # center is between x and x+1
    print(np.average(hist2[1][x:x + 2]), np.average(hist2[2][y:y + 2]))

In [ ]:
(df[h][f"{probe}_Raw_Bmag"]/df[h]["NMR [T]"]).describe()

In [ ]:
df[h][f"{probe}_Cal_T"].describe()

In [ ]:
fig = scatter2d(df[stable_temp].index, df[stable_temp]["Floor"], inline=False)

In [ ]:
fig = scatter2d(df.index, df["Floor"].diff(5).rolling(5).mean(), inline=False)

In [ ]:
df["Roof"].diff(4).plot()

In [ ]:
df["NMR [T]"].plot()
#df["Roof"].diff(1).plot();

In [47]:
p_low = [df[l][f"{probe}_Cal_T"].mean(), (df[l][f"{probe}_Raw_Bmag"]/df[l]["NMR [T]"]).mean()]
p_high = [df[h][f"{probe}_Cal_T"].mean(), (df[h][f"{probe}_Raw_Bmag"]/df[h]["NMR [T]"]).mean()]
p_mid = [df[m][f"{probe}_Cal_T"].mean(), (df[m][f"{probe}_Raw_Bmag"]/df[m]["NMR [T]"]).mean()]

In [48]:
std_low = [df[l][f"{probe}_Cal_T"].std(), (df[l][f"{probe}_Raw_Bmag"]/df[l]["NMR [T]"]).std()]
std_high = [df[h][f"{probe}_Cal_T"].std(), (df[h][f"{probe}_Raw_Bmag"]/df[h]["NMR [T]"]).std()]
std_mid = [df[m][f"{probe}_Cal_T"].std(), (df[m][f"{probe}_Raw_Bmag"]/df[m]["NMR [T]"]).std()]

In [49]:
std_low

[0.021367943656077457, 54.51474064649545]

In [52]:
fig = scatter2d(df[stable_temp][f"{probe}_Cal_T"],
                df[stable_temp][f"{probe}_Raw_Bmag"] / df[stable_temp]["NMR [T]"],
                df[stable_temp].days_delta, show_plot=False)
#d = [df[f"{probe}_Raw_X"].min(), df[f"{probe}_Raw_X"].max()]
fig.add_trace(go.Scatter(
                    x=df[stable_temp][f"{probe}_Cal_T"], y=df[stable_temp]["Bmag_lsq"],
                    mode='lines', name="unity",
                    line=dict(color='deeppink', width=2,
                              dash='dash')));
#plot(fig)

fig.update_traces(marker=dict(size=5));
#plot(fig);

In [53]:
fig.add_trace(go.Scatter(
    x= [p_low[0],p_mid[0],p_high[0]],
    y=[p_low[1],p_mid[1],p_high[1]],
    mode='markers',
    error_x=dict(
        type='data',
        array=[std_low[0], std_mid[0], std_high[0]],
        visible=True),
    error_y=dict(
        type='data',
        array=[std_low[1], std_mid[1], std_high[1]],
        visible=True),
    marker=dict(
        size=6,
        color="Red",
    )
))

#fig = go.Figure(traces)
plot(fig, filename=plotdir+"temp_fit_2point_aggregate.html")

'/home/ckampa/Coding/hallprobecalib_extras/plots/hall_probe_stability/2019-12-04/temp_fit_2point_aggregate.html'

In [54]:
pio.write_image(fig, plotdir+"temp_fit_2point_aggregate.pdf")
pio.write_image(fig, plotdir+"temp_fit_2point_aggregate.png")